# 🎓 AI 질문 자동 생성기 (책 기반)

이 노트북은 책 제목을 입력하면 ChatGPT API를 사용해 **에세이형 토론 질문 22개**를 자동 생성하고, Google Sheets에 저장하는 워크플로우입니다.

📌 이 노트북은 다음을 포함합니다:
- OpenAI API 설정
- 책 제목 입력
- GPT-3.5로 질문 생성
- pandas로 정리
- Google Sheets 자동 업로드

In [ ]:
# ✅ 필요한 라이브러리 설치 (최초 1회 실행)
!pip install --upgrade openai gspread oauth2client pandas --quiet

## 🔐 OpenAI API 키 입력
- 아래 코드에 본인의 API 키(`sk-...`)를 넣으세요.
- GPT-4 접근권한이 없을 경우 GPT-3.5로 자동 생성됩니다.

In [ ]:
from openai import OpenAI
import pandas as pd

# 여기에 본인의 OpenAI API 키 입력
client = OpenAI(api_key="")

## 📁 Google Sheets 인증파일 업로드
- Google Cloud Console에서 `credentials.json` 다운로드 받아 업로드하세요.
- 서비스 계정 이메일을 Google Sheets 문서에 공유해야 합니다.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving credentials.json to credentials (2).json


## 📚 분석할 책 목록 정의
- 책 제목과 저자 이름을 리스트 형식으로 입력합니다.
- 여러 권 입력 가능

In [ ]:
books = [
    {"title": "To Kill a Mockingbird", "author": "Harper Lee"},
    {"title": "The Giver", "author": "Lois Lowry"}
]

## ✨ GPT-3.5를 사용한 질문 생성 함수
- 에세이형, 비판적 사고 기반 질문 22개 생성
- 1~22번 형식으로 출력된 질문만 추출

In [ ]:
def generate_questions(book):
    prompt = f"""
You are an experienced curriculum designer for middle and high school humanities classes.
Please generate 22 unique, high-quality essay-style discussion questions based on the book titled \"{book['title']}\" by {book['author']}. Questions should promote deep thinking, character analysis, and real-world themes. Format:
1. ...
2. ...
...
22. ...
"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )
        content = response.choices[0].message.content
        return [line.strip()[3:].strip() for line in content.split('\n') if line.strip().startswith(tuple("123456789"))]
    except Exception as e:
        print("⚠️ 오류 발생:", e)
        return []

## 📊 질문 생성 및 DataFrame 저장
- 각 책에 대해 질문 생성
- pandas DataFrame으로 정리하고 csv로 저장

In [ ]:
all_questions = []
for book in books:
    qlist = generate_questions(book)
    for i, q in enumerate(qlist):
        all_questions.append({
            "book_title": book['title'],
            "author": book['author'],
            "question_number": i+1,
            "question": q,
            "length": len(q),
            "source": "AI"
        })

df = pd.DataFrame(all_questions)
df.to_csv("ai_questions.csv", index=False)
df.head()

,book_title,author,question_number,question,length,source
0,To Kill a Mockingbird,Harper Lee,1,How does the character of Atticus Finch challe...,98,AI
1,To Kill a Mockingbird,Harper Lee,2,Discuss the role of race and racism in Maycomb...,81,AI
2,To Kill a Mockingbird,Harper Lee,3,How does Harper Lee use the character of Boo R...,86,AI
3,To Kill a Mockingbird,Harper Lee,4,Analyze the character development of Scout thr...,94,AI
4,To Kill a Mockingbird,Harper Lee,5,How does the trial of Tom Robinson highlight t...,81,AI


from matplotlib import pyplot as plt
_df_0['question_number'].plot(kind='hist', bins=20, title='question_number')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['length'].plot(kind='hist', bins=20, title='length')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('question').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3.plot(kind='scatter', x='question_number', y='length', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7['length'].plot(kind='line', figsize=(8, 4), title='length')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_8['question'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_8, x='question_number', y='question', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_9['question'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_9, x='length', y='question', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

## 📤 Google Sheets에 자동 업로드
- 먼저 시트를 만들어야 합니다 (예: `AI_Questions_Dataset` → Sheet1 탭)
- 서비스 계정 이메일을 공유해야 합니다.

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
gc = gspread.authorize(creds)

sheet = gc.open("AI_Questions_Dataset").worksheet("Sheet1")
sheet.clear()
sheet.update([df.columns.values.tolist()] + df.values.tolist())
print("✅ Google Sheets 업로드 완료")

✅ Google Sheets 업로드 완료
